<a href="https://colab.research.google.com/github/dietmarja/LLM-Elements/blob/main/vector_db/pinecone_02a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
%run setup_colab.py


In [77]:
%run setup_pinecone.py

Existing index deleted
Index created successfully


In [85]:
!pip -qU install transformers pinecone-client torch langchain PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 50.3 MB/s eta 0:00:00


In [86]:
import fitz  # PyMuPDF
from langchain import LLMChain
from transformers import AutoModel, AutoTokenizer
import pinecone
import torch


In [103]:
# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec



# Create a pincecone client
pc = Pinecone(api_key=pinecone_api_key)

# Create a pincecone client
index_name = "docs-quickstart-index"

# Check if the index exists and delete it if it does
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print("Existing index deleted")

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",  # Available metrics: "euclidean"/"manhattan"/"dotproduct"/"cosine"
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

print("Index created successfully")

# Connect pc to the index
index = pc.Index(index_name)
print("Connected to the index")

Existing index deleted
Index created successfully
Connected to the index


In [80]:
# Load a Hugging Face model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a simple embedding function using Hugging Face model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()


In [81]:
#!pip install -qU transformers pinecone-client torch langchain

In [88]:
from transformers import AutoModel, AutoTokenizer
import pinecone
import torch

# Load a Hugging Face model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Define a simple embedding function using Hugging Face model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze().tolist()
        # Ensure the embedding is a flat list of floats
        if isinstance(embeddings[0], list):
            embeddings = [item for sublist in embeddings for item in sublist]
    if len(embeddings) != 384:
        raise ValueError(f"Embedding dimension mismatch: Expected 384, got {len(embeddings)}")
    return embeddings


In [92]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Function to store document in Pinecone
def store_document(doc_id, text):
    embedding = embed_text(text)
    index.upsert(vectors=[(doc_id, embedding)])

# Path to PDF file
pdf_path_1 = "Attention_Paper.pdf"
#pdf_path_2 = "path/to/your/document2.pdf"

# Extract text from PDFs
text_1 = extract_text_from_pdf(pdf_path_1)
#text_2 = extract_text_from_pdf(pdf_path_2)

# Store extracted text in Pinecone
store_document("doc_1", text_1)
#store_document("doc_2", text_2)

# Query Pinecone for multiple documents
query_embedding_1 = embed_text("search query for doc 1")
#query_embedding_2 = embed_text("search query for doc 2")

# Show the query embedding
print(f"Query embedding 1: {query_embedding_1}")
print(f"Query embedding 1 dimension: {len(query_embedding_1)}")
#print(f"Query embedding 2: {query_embedding_2}")
#print(f"Query embedding 2 dimension: {len(query_embedding_2)}")

results_1 = index.query(vector=query_embedding_1, top_k=10)
#results_2 = index.query(vector=query_embedding_2, top_k=10)

print("Results for document 1:")
print(results_1)

#print("\nResults for document 2:")
#print(results_2)









Query embedding 1: [-0.14562642574310303, 0.5601049661636353, 0.5747165679931641, 0.5388968586921692, 0.0780593603849411, 0.31727224588394165, -0.12419245392084122, 0.6555048227310181, 0.35049957036972046, 0.0017658972647041082, 0.09825917333364487, 0.16016599535942078, 0.6524692177772522, -0.4563508629798889, -0.5212566256523132, -0.10932393372058868, -0.3860594928264618, 0.05149644613265991, 0.14169219136238098, 0.2924876809120178, 0.17130199074745178, 0.7985489964485168, 0.09196789562702179, -0.3987356722354889, -0.02856002189218998, -0.07411689311265945, -0.5267559885978699, -0.9285850524902344, -0.12639212608337402, 0.013689696788787842, -0.07382040470838547, 0.178280308842659, 0.2382897287607193, 0.6428471803665161, 0.43961596488952637, -0.3523942530155182, 0.07005397230386734, -0.3105872571468353, 0.3699536919593811, 0.2713792026042938, -0.2558605968952179, -0.26600202918052673, 0.09510897099971771, -0.3179085850715637, 0.18635237216949463, -0.09240924566984177, -0.5089095830917

In [102]:
import fitz  # PyMuPDF
from transformers import AutoModel, AutoTokenizer
import pinecone
import torch

# Load a Hugging Face model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a simple embedding function using Hugging Face model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze().tolist()
        # Ensure the embedding is a flat list of floats
        if isinstance(embeddings[0], list):
            embeddings = [item for sublist in embeddings for item in sublist]
    if len(embeddings) != 384:
        raise ValueError(f"Embedding dimension mismatch: Expected 384, got {len(embeddings)}")
    return embeddings

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    texts = []
    with fitz.open(pdf_path) as doc:
        for page_num, page in enumerate(doc):
            text = page.get_text()
            texts.append((page_num, text))
    return texts

# Function to store document in Pinecone
def store_document(doc_id, text):
    embedding = embed_text(text)
    index.upsert(vectors=[(doc_id, embedding)])

# Path to your PDF file
pdf_path_1 = "Attention_Paper.pdf"
pdf_path_2 = "Lora_Paper.pdf"

# Extract text from PDF
texts_1 = extract_text_from_pdf(pdf_path_1)
texts_2 = extract_text_from_pdf(pdf_path_2)


# Store each page as a separate document in Pinecone
for page_num, text in texts_1:
    store_document(f"doc_1_page_{page_num}", text)

for page_num, text in texts_1:
    store_document(f"doc_2_page_{page_num}", text)


# Query Pinecone for a document
query_embedding = embed_text("search query for the Attention paper")
results = index.query(vector=query_embedding, top_k=10)

print("Results:")
print(results)


Results:
{'matches': [{'id': 'doc_1_page_3', 'score': 0.388306588, 'values': []},
             {'id': 'doc_2_page_3', 'score': 0.388306588, 'values': []},
             {'id': 'doc_1_page_10', 'score': 0.382295758, 'values': []},
             {'id': 'doc_2_page_6', 'score': 0.378480464, 'values': []},
             {'id': 'doc_1_page_6', 'score': 0.378480464, 'values': []},
             {'id': 'doc_1_page_2', 'score': 0.359783769, 'values': []},
             {'id': 'doc_2_page_2', 'score': 0.359783769, 'values': []},
             {'id': 'doc_2_page_4', 'score': 0.358001769, 'values': []},
             {'id': 'doc_1_page_4', 'score': 0.358001769, 'values': []},
             {'id': 'doc_1_page_0', 'score': 0.34228003, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}


In [ ]:
# Storing text strings

## Function to store document in Pinecone
#def store_document(doc_id, text):
#    embedding = embed_text(text)
#    index.upsert(vectors=[(doc_id, embedding)])

# Store multiple strings
store_document("doc_1", "This is the wonderful document 1.")
store_document("doc_2", "This is the incredible document 2.")

# Query Pinecone for multiple documents
query_embedding_1 = embed_text("search query for doc 1")
query_embedding_2 = embed_text("search query for doc 2")

# Show the query embedding
print(f"Query embedding 1: {query_embedding_1}")
print(f"Query embedding 1 dimension: {len(query_embedding_1)}")
print(f"Query embedding 2: {query_embedding_2}")
print(f"Query embedding 2 dimension: {len(query_embedding_2)}")

#print(index.describe_index_stats())


In [ ]:
results_1 = index.query(vector=query_embedding_1, top_k=10)
results_2 = index.query(vector=query_embedding_2, top_k=10)

print(results_2)
print(results_1)
